<a href="https://colab.research.google.com/github/lauberto/uni_mydirectory/blob/master/compling/homework/HW8_andrea_grillandi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [0]:
PATH_TO_DATA = "https://github.com/mannefedov/ru_kw_eval_datasets/tree/master/data"

In [8]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

FileNotFoundError: ignored